### <font color = 'blue'> Project Data collection
### <font color = 'blue'> Mairame Samba NIANG              
### <font color = 'blue'> Coopérative:Data Science

In [85]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

In [86]:
#first url chiens

### <font color = 'blue'> Collect data over one page

In [87]:
# url
url_1 = 'https://sn.coinafrique.com/categorie/chiens'
# get the code html of the page using the get function requests
res = get(url_1)
# stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.content, 'html.parser')

In [88]:
# get all containers 
containers = soup.find_all('div', 'col s6 m4 l3')
# length de containers
len(containers)

84

In [89]:
# scrape data from the first container
container= containers[0]
#scrape the following informations: 
Name=container.find('p', 'ad__card-description').a.text
#print(Name)
#get price
price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
#print(price)
#get adress
adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
#print(adress)
# get image_link
img = container.find('img')
img_link = img['src']
#print(img_link)

In [90]:
# scrape data from all the containers
import pandas as pd
data = []
for container in containers:
    try:
        #scrape the following informations: 
        Name=container.find('p', 'ad__card-description').a.text
        #get price
        price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
        #get adress
        adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
        # get image_link
        img = container.find('img')
        img_link = img['src']
        dic={
            "Name":Name,
            "price":price,
            "adress":adress,
            "img_link":img_link
        }
        data.append(dic)
        
    except:
        pass
DF=pd.DataFrame(data)

In [91]:
DF.head()

,Name,price,adress,img_link
0,Berger malinois charbonner,250,"Thies,Sénégal",https://images.coinafrique.com/thumb_5649558_u...
1,Chiot Bichon Maltais,100000,"Almadies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5644579_u...
2,Chiots Labradors Retriever,300000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5640905_u...
3,Chiot Malinois,200000,"Dakar,Sénégal",https://images.coinafrique.com/thumb_5628892_u...
4,Chiot Berger Malinois,150000,"Almadies2,Dakar,Sénégal",https://images.coinafrique.com/thumb_5628715_u...


In [92]:
DF.shape

(84, 4)

### <font color = 'blue'> Collect data over multiple pages

In [93]:
# create a empty dataframe df
def scrape_pages(n_pages):
    
    df = pd.DataFrame()
    # loop over pages indexes
    for index in range(1, n_pages + 1 ):
        url_1 = f'https://sn.coinafrique.com/categorie/chiens?page={index}'
        # get the code html of the page using the get function requests
        res = get(url_1)
        # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
        soup = bs(res.content, 'html.parser')
        containers = soup.find_all('div', 'col s6 m4 l3')
        data = []
        for container in containers:
            try:
                #scrape the following informations: 
                Name=container.find('p', 'ad__card-description').a.text
                #get price
                price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
                #get adress
                adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
                # get image_link
                img = container.find('img')
                img_link = img['src']
                dic={
                    "Name":Name,
                    "price":price,
                    "adress":adress,
                    "img_link":img_link
                }
                data.append(dic)

            except:
                pass
        DF=pd.DataFrame(data)
        df=pd.concat([df,DF],axis=0).reset_index(drop=True)
    return df

In [94]:
df=scrape_pages(10)
df.head()

,Name,price,adress,img_link
0,Berger malinois charbonner,250,"Thies,Sénégal",https://images.coinafrique.com/thumb_5649558_u...
1,Chiot Bichon Maltais,100000,"Almadies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5644579_u...
2,Chiots Labradors Retriever,300000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5640905_u...
3,Chiot Malinois,200000,"Dakar,Sénégal",https://images.coinafrique.com/thumb_5628892_u...
4,Chiot Berger Malinois,150000,"Almadies2,Dakar,Sénégal",https://images.coinafrique.com/thumb_5628715_u...


In [95]:
df.shape

(840, 4)

In [96]:
df.dtypes

Name        object
price       object
adress      object
img_link    object
dtype: object

In [97]:
#check the infos of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      840 non-null    object
 1   price     840 non-null    object
 2   adress    840 non-null    object
 3   img_link  840 non-null    object
dtypes: object(4)
memory usage: 26.4+ KB


In [99]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [102]:
df['price'] = df['price'].astype('Int64')

In [103]:
print(df.dtypes)

Name        object
price        Int64
adress      object
img_link    object
dtype: object


In [62]:
# Clean the data

In [104]:
print(df.isnull().sum())

Name          0
price       175
adress        0
img_link      0
dtype: int64


In [105]:
df.isna().any().sum()

np.int64(1)

In [106]:
df['price'].fillna(df['price'].median(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4872\1576110267.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].median(), inplace=True)


In [107]:
print(df.isnull().sum())

Name        0
price       0
adress      0
img_link    0
dtype: int64


In [108]:
print(df.duplicated().sum())

0


In [109]:
df.duplicated().sum()

np.int64(0)

In [110]:
df.shape

(840, 4)

In [70]:
#scraped data in a sql database.

In [71]:
#!pip install selenium

In [72]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs
import pandas as pd

In [73]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [74]:
# import sqlite3
import sqlite3
import pandas as pd
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('renting_animaux.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
#c.execute('''CREATE TABLE t1_table(Name,price,adress,img_link)''')

In [75]:
#insert informations in the table 
df.to_sql('t1_table', conn, if_exists='append',index=False)
#charge the change
conn.commit()

In [76]:
# import the table from renting_rooms.db
df_sql = pd.read_sql_query('SELECT * FROM t1_table', conn)

In [77]:
df_sql.head()

,Name,price,adress,img_link
0,Berger malinois charbonner,250.0,"Thies,Sénégal",https://images.coinafrique.com/thumb_5649558_u...
1,Chiot Bichon Maltais,100000.0,"Almadies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5644579_u...
2,Chiots Labradors Retriever,300000.0,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5640905_u...
3,Chiot Malinois,200000.0,"Dakar,Sénégal",https://images.coinafrique.com/thumb_5628892_u...
4,Chiot Berger Malinois,150000.0,"Almadies2,Dakar,Sénégal",https://images.coinafrique.com/thumb_5628715_u...


In [78]:
df.shape

(840, 4)

In [80]:
df.dtypes

Name         object
price       float64
adress       object
img_link     object
dtype: object

In [79]:
print(df.isnull().sum())

Name        0
price       0
adress      0
img_link    0
dtype: int64


# second url moutons

### <font color = 'blue'> Collect data over one page

In [112]:
# url
url = 'https://sn.coinafrique.com/categorie/moutons'
# get the code html of the page using the get function requests
res = get(url)
# stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.content, 'html.parser')

In [113]:
# get all containers 
containers = soup.find_all('div', 'col s6 m4 l3')
# length de containers
len(containers)

84

In [114]:
# scrape data from the first container
container= containers[0]
#scrape the following informations: 
#brand, model, year,reference, kms driven, adress, price,img_link
name=container.find('p', 'ad__card-description').text
#print(name)
#get price
price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
#print(price)
#get adress
adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
#print(adress)
# get image_link
img = container.find('img')
img_link = img['src']
#print(img_link)

In [115]:
# scrape data from all the containers
import pandas as pd
data = []
for container in containers:
    try:
        #scrape the following informations: 
        Name=container.find('p', 'ad__card-description').a.text
        #get price
        price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
        #get adress
        adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
        # get image_link
        img = container.find('img')
        img_link = img['src']
        dic={
            "Name":Name,
            "price":price,
            "adress":adress,
            "img_link":img_link
        }
        data.append(dic)
        
    except:
        pass
DF=pd.DataFrame(data)

In [116]:
DF.head()

,Name,price,adress,img_link
0,Fauteuil pliable,50000,"ParcelleAssainies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5641122_u...
1,Agneau pur sang,Prixsurdemande,"Mbao,Dakar,Sénégal",https://images.coinafrique.com/thumb_5631456_u...
2,Mouton,125000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630109_u...
3,Mouton Ladoum,800000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630102_u...
4,Moutons Ladoum,350000,"Thies,Sénégal",https://images.coinafrique.com/thumb_5613540_u...


In [117]:
DF.shape

(84, 4)

### <font color = 'blue'> Collect data over multiple pages

In [212]:
# create a empty dataframe df
def scrape_pages(n_pages):
    
    df = pd.DataFrame()
    # loop over pages indexes
    for index in range(1, n_pages + 1 ):
        url = f'https://sn.coinafrique.com/categorie/moutons?page={index}'
        # get the code html of the page using the get function requests
        res = get(url)
        # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
        soup = bs(res.content, 'html.parser')
        containers = soup.find_all('div', 'col s6 m4 l3')
        data = []
        for container in containers:
            try:
                #scrape the following informations: 
                Name=container.find('p', 'ad__card-description').a.text
                #get price
                price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
                #get adress
                adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
                # get image_link
                img = container.find('img')
                img_link = img['src']
                dic={
                    "Name":Name,
                    "price":price,
                    "adress":adress,
                    "img_link":img_link
                }
                data.append(dic)

            except:
                pass
        DF=pd.DataFrame(data)
        df=pd.concat([df,DF],axis=0).reset_index(drop=True)
    return df

In [213]:
df=scrape_pages(10)
df.head()

,Name,price,adress,img_link
0,Fauteuil pliable,50000,"ParcelleAssainies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5641122_u...
1,Agneau pur sang,Prixsurdemande,"Mbao,Dakar,Sénégal",https://images.coinafrique.com/thumb_5631456_u...
2,Mouton,125000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630109_u...
3,Mouton Ladoum,800000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630102_u...
4,Moutons Ladoum,350000,"Thies,Sénégal",https://images.coinafrique.com/thumb_5613540_u...


In [214]:
df.shape

(840, 4)

In [215]:
df.dtypes

Name        object
price       object
adress      object
img_link    object
dtype: object

In [216]:
#check the infos of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      840 non-null    object
 1   price     840 non-null    object
 2   adress    840 non-null    object
 3   img_link  840 non-null    object
dtypes: object(4)
memory usage: 26.4+ KB


In [217]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [218]:
df['price'] = df['price'].astype('Int64')

In [219]:
print(df.dtypes)

Name        object
price        Int64
adress      object
img_link    object
dtype: object


In [220]:
#clean data

In [221]:
print(df.isnull().sum())

Name          0
price       140
adress        0
img_link      0
dtype: int64


In [222]:
df.isna().any().sum()

np.int64(1)

In [223]:
df['price'].fillna(df['price'].median(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4872\1576110267.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].median(), inplace=True)


In [224]:
print(df.isnull().sum())

Name        0
price       0
adress      0
img_link    0
dtype: int64


In [225]:
print(df.duplicated().sum())

0


In [132]:
df.duplicated().sum()

np.int64(0)

In [226]:
df.shape

(840, 4)

In [134]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs
import pandas as pd

In [135]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [136]:
# import sqlite3
import sqlite3
import pandas as pd
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('renting_animaux.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
#c.execute('''CREATE TABLE t1_table(Name,price,adress,img_link)''')

In [137]:
#insert informations in the table 
df.to_sql('t2_table', conn, if_exists='append',index=False)
#charge the change
conn.commit()

In [138]:
# import the table from renting_rooms.db
df_sql = pd.read_sql_query('SELECT * FROM t2_table', conn)

In [139]:
df_sql.head()

,Name,price,adress,img_link
0,Fauteuil pliable,50000,"ParcelleAssainies,Dakar,Sénégal",https://images.coinafrique.com/thumb_5641122_u...
1,Agneau pur sang,250000,"Mbao,Dakar,Sénégal",https://images.coinafrique.com/thumb_5631456_u...
2,Mouton,125000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630109_u...
3,Mouton Ladoum,800000,"Guediawaye,Dakar,Sénégal",https://images.coinafrique.com/thumb_5630102_u...
4,Moutons Ladoum,350000,"Thies,Sénégal",https://images.coinafrique.com/thumb_5613540_u...


In [140]:
df.shape

(840, 4)

# third url lapins-et- pigeons

### <font color = 'blue'> Collect data over one page

In [141]:
# url
url = 'https://sn.coinafrique.com/categorie/poules-lapins-et-pigeons'
# get the code html of the page using the get function requests
res = get(url)
# stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.content, 'html.parser')
# get all containers 
containers = soup.find_all('div', 'col s6 m4 l3')
# length de containers
len(containers)

84

In [142]:
# scrape data from the first container
container= containers[0]
#scrape the following informations: 
#brand, model, year,reference, kms driven, adress, price,img_link
details=container.find('p', 'ad__card-description').text
print(details)
#get price
price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
print(price)
#get adress
adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
print(adress)
# get image_link
img = container.find('img')
img_link = img['src']
print(img_link)

Poulettes pré ponte de 124 jours
5000
Dakar,Sénégal
https://images.coinafrique.com/thumb_5655142_uploaded_image2_1764832679.jpeg


In [143]:
# scrape data from all the containers
import pandas as pd
data = []
for container in containers:
    try:
        #scrape the following informations: 
        Name=container.find('p', 'ad__card-description').a.text
        #get price
        price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
        #get adress
        adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
        # get image_link
        img = container.find('img')
        img_link = img['src']
        dic={
            "Name":Name,
            "price":price,
            "adress":adress,
            "img_link":img_link
        }
        data.append(dic)
        
    except:
        pass
DF=pd.DataFrame(data)

### <font color = 'blue'> Collect data over multiple pages

In [162]:
# create a empty dataframe df
def scrape_pages(n_pages):
    
    df = pd.DataFrame()
    # loop over pages indexes
    for index in range(1, n_pages + 1 ):
        url = f'https://sn.coinafrique.com/categorie/poules-lapins-et-pigeons?page={index}'
        # get the code html of the page using the get function requests
        res = get(url)
        # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
        soup = bs(res.content, 'html.parser')
        containers = soup.find_all('div', 'col s6 m4 l3')
        data = []
        for container in containers:
            try:
                #scrape the following informations: 
                Name=container.find('p', 'ad__card-description').a.text
                #get price
                price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
                #get adress
                adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
                # get image_link
                img = container.find('img')
                img_link = img['src']
                dic={
                    "Name":Name,
                    "price":price,
                    "adress":adress,
                    "img_link":img_link
                }
                data.append(dic)

            except:
                pass
        DF=pd.DataFrame(data)
        df=pd.concat([df,DF],axis=0).reset_index(drop=True)
    return df

In [163]:
df=scrape_pages(10)
df.head()

,Name,price,adress,img_link
0,Poulettes pré ponte de 124 jours,5000,"Dakar,Sénégal",https://images.coinafrique.com/thumb_5655142_u...
1,Poussin goliath,300,"Thies,Sénégal",https://images.coinafrique.com/thumb_5648409_u...
2,Poulet Goliath,Prixsurdemande,"Mbao,Dakar,Sénégal",https://images.coinafrique.com/thumb_5648319_u...
3,Pigeon Voyageur,20000,"Rufisque,Sénégal",https://images.coinafrique.com/thumb_5642510_u...
4,Poule Brahman,Prixsurdemande,"KeurMassar,Sénégal",https://images.coinafrique.com/thumb_5594420_u...


In [166]:
df.shape

(803, 4)

In [167]:
df.dtypes

Name        object
price       object
adress      object
img_link    object
dtype: object

In [168]:
#check the infos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      803 non-null    object
 1   price     803 non-null    object
 2   adress    803 non-null    object
 3   img_link  803 non-null    object
dtypes: object(4)
memory usage: 25.2+ KB


In [169]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [170]:
df['price'] = df['price'].astype('Int64')

In [171]:
print(df.dtypes)

Name        object
price        Int64
adress      object
img_link    object
dtype: object


In [172]:
#clean data

In [173]:
print(df.isnull().sum())

Name          0
price       178
adress        0
img_link      0
dtype: int64


In [174]:
df.isna().any().sum()

np.int64(1)

In [175]:
df['price'].fillna(df['price'].median(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4872\1576110267.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].median(), inplace=True)


In [176]:
print(df.isnull().sum())

Name        0
price       0
adress      0
img_link    0
dtype: int64


In [177]:
print(df.duplicated().sum())

0


In [178]:
df.shape

(803, 4)

In [179]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs
import pandas as pd

In [180]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [181]:
# import sqlite3
import sqlite3
import pandas as pd
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('renting_animaux.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
#c.execute('''CREATE TABLE t1_table(Name,price,adress,img_link)''')

In [182]:
#insert informations in the table 
df.to_sql('t3_table', conn, if_exists='append',index=False)
#charge the change
conn.commit()

In [183]:
# import the table from renting_rooms.db
df_sql = pd.read_sql_query('SELECT * FROM t3_table', conn)

In [184]:
df_sql.head()

,Name,price,adress,img_link
0,Poulettes pré ponte de 124 jours,5000,"Dakar,Sénégal",https://images.coinafrique.com/thumb_5655142_u...
1,Poussin goliath,300,"Thies,Sénégal",https://images.coinafrique.com/thumb_5648409_u...
2,Poulet Goliath,25000,"Mbao,Dakar,Sénégal",https://images.coinafrique.com/thumb_5648319_u...
3,Pigeon Voyageur,20000,"Rufisque,Sénégal",https://images.coinafrique.com/thumb_5642510_u...
4,Poule Brahman,25000,"KeurMassar,Sénégal",https://images.coinafrique.com/thumb_5594420_u...


In [185]:
df.shape

(803, 4)

In [84]:
#four url autres-animaux

### <font color = 'blue'> Collect data over one page

In [186]:
# url
url = 'https://sn.coinafrique.com/categorie/autres-animaux'
# get the code html of the page using the get function requests
res = get(url)
# stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.content, 'html.parser')
# get all containers 
containers = soup.find_all('div', 'col s6 m4 l3')
# length de containers
len(containers)

84

In [187]:
# scrape data from the first container
container= containers[0]
#scrape the following informations: 
#brand, model, year,reference, kms driven, adress, price,img_link
name=container.find('p', 'ad__card-description').text
print(name)
#get price
price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
print(price)
#get adress
adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
print(adress)
# get image_link
img = container.find('img')
img_link = img['src']
print(img_link)

Bœufs
Prixsurdemande
Mermoz-SacréCoeur,Dakar,Sénégal
https://images.coinafrique.com/thumb_5642438_uploaded_image1_1764168383.jpg


In [188]:
# scrape data from all the containers
import pandas as pd
data = []
for container in containers:
    try:
        #scrape the following informations: 
        Name=container.find('p', 'ad__card-description').a.text
        #get price
        price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
        #get adress
        adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
        # get image_link
        img = container.find('img')
        img_link = img['src']
        dic={
            "Name":Name,
            "price":price,
            "adress":adress,
            "img_link":img_link
        }
        data.append(dic)
        
    except:
        pass
DF=pd.DataFrame(data)

In [189]:
DF.head()

,Name,price,adress,img_link
0,Bœufs,Prixsurdemande,"Mermoz-SacréCoeur,Dakar,Sénégal",https://images.coinafrique.com/thumb_5642438_u...
1,Taureau,2500000,"Fatick,Sénégal",https://images.coinafrique.com/thumb_5634272_u...
2,Bœuf,650,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632019_u...
3,Bœuf,725,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632012_u...
4,Vache,350000,"Patted'oie,Dakar,Sénégal",https://images.coinafrique.com/thumb_5587581_u...


In [190]:
DF.shape

(84, 4)

# <font color = 'blue'> Collect data over multiple pages

In [191]:
# create a empty dataframe df
def scrape_pages(n_pages):
    
    df = pd.DataFrame()
    # loop over pages indexes
    for index in range(1, n_pages + 1 ):
        url = f'https://sn.coinafrique.com/categorie/autres-animaux?page={index}'
        # get the code html of the page using the get function requests
        res = get(url)
        # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
        soup = bs(res.content, 'html.parser')
        containers = soup.find_all('div', 'col s6 m4 l3')
        data = []
        for container in containers:
            try:
                #scrape the following informations: 
                Name=container.find('p', 'ad__card-description').a.text
                #get price
                price="".join(container.find('p', 'ad__card-price').text.strip().split()).replace('CFA', '')
                #get adress
                adress= "".join(container.find('p', 'ad__card-location').text.strip().split()).replace("location_on","")
                # get image_link
                img = container.find('img')
                img_link = img['src']
                dic={
                    "Name":Name,
                    "price":price,
                    "adress":adress,
                    "img_link":img_link
                }
                data.append(dic)

            except:
                pass
        DF=pd.DataFrame(data)
        df=pd.concat([df,DF],axis=0).reset_index(drop=True)
    return df

In [192]:
df=scrape_pages(10)
df.head()

,Name,price,adress,img_link
0,Bœufs,Prixsurdemande,"Mermoz-SacréCoeur,Dakar,Sénégal",https://images.coinafrique.com/thumb_5642438_u...
1,Taureau,2500000,"Fatick,Sénégal",https://images.coinafrique.com/thumb_5634272_u...
2,Bœuf,650,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632019_u...
3,Bœuf,725,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632012_u...
4,Vache,350000,"Patted'oie,Dakar,Sénégal",https://images.coinafrique.com/thumb_5587581_u...


In [193]:
df.shape

(490, 4)

In [194]:
df.dtypes

Name        object
price       object
adress      object
img_link    object
dtype: object

In [195]:
#cheks the infos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      490 non-null    object
 1   price     490 non-null    object
 2   adress    490 non-null    object
 3   img_link  490 non-null    object
dtypes: object(4)
memory usage: 15.4+ KB


In [196]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [197]:
df['price'] = df['price'].astype('Int64')

In [198]:
print(df.dtypes)

Name        object
price        Int64
adress      object
img_link    object
dtype: object


In [199]:
#clean data

In [200]:
print(df.isnull().sum())

Name          0
price       127
adress        0
img_link      0
dtype: int64


In [201]:
df['price'].fillna(df['price'].median(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_4872\1576110267.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].median(), inplace=True)


In [202]:
print(df.isnull().sum())

Name        0
price       0
adress      0
img_link    0
dtype: int64


In [203]:
print(df.duplicated().sum())

0


In [204]:
df.shape

(490, 4)

In [205]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs
import pandas as pd

In [206]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [207]:
# import sqlite3
import sqlite3
import pandas as pd
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('renting_animaux.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
#c.execute('''CREATE TABLE t1_table(Name,price,adress,img_link)''')

In [208]:
#insert informations in the table 
df.to_sql('t4_table', conn, if_exists='append',index=False)
#charge the change
conn.commit()

In [209]:
# import the table from renting_rooms.db
df_sql = pd.read_sql_query('SELECT * FROM t4_table', conn)

In [210]:
df_sql.head()

,Name,price,adress,img_link
0,Bœufs,90000,"Mermoz-SacréCoeur,Dakar,Sénégal",https://images.coinafrique.com/thumb_5642438_u...
1,Taureau,2500000,"Fatick,Sénégal",https://images.coinafrique.com/thumb_5634272_u...
2,Bœuf,650,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632019_u...
3,Bœuf,725,"Bambey,Sénégal",https://images.coinafrique.com/thumb_5632012_u...
4,Vache,350000,"Patted'oie,Dakar,Sénégal",https://images.coinafrique.com/thumb_5587581_u...


In [211]:
df.shape

(490, 4)